In [2]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras.models import Sequential, load_model

from keras_self_attention import SeqSelfAttention
from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import keras
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *
from keras.models import model_from_json
import json



Using TensorFlow backend.


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/PAWS_Wiki/train.tsv'

In [6]:
ppdb_Tb=Table.read_table("dataset/ppdb/sanitized.tsv")

/usr/local/lib/python3.5/dist-packages/datascience/tables.py:132: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  df = pandas.read_table(filepath_or_buffer, *args, **vargs)


In [6]:
test_Tb=Table.read_table('dataset/PAWS_Wiki/test.tsv')

FileNotFoundError: [Errno 2] File b'dataset/PAWS_Wiki/test.tsv' does not exist: b'dataset/PAWS_Wiki/test.tsv'

In [7]:
train_id = train_Tb.column("id")
train_sen1=train_Tb.column("sentence1")
train_sen2=train_Tb.column("sentence2")  #PAWS_QQP
train_row = train_Tb.num_rows
dev_id=dev_Tb.column("id")
dev_row=dev_Tb.num_rows
dev_sen1=dev_Tb.column("sentence1")
dev_sen2=dev_Tb.column("sentence2")
test_id=test_Tb.column("id")
test_row=test_Tb.num_rows
test_sen1=test_Tb.column("sentence1")
test_sen2=test_Tb.column("sentence2")
train_label = train_Tb.column("label")
dev_label=dev_Tb.column("label")
test_label=test_Tb.column("label")

phrase = ppdb_Tb.column("PHRASE")
paraphrase=ppdb_Tb.column("PARAPHRASE")   #ppdb


In [ ]:
index_list=[]
PPDB_id = []
sent1=[]
sent2=[]
para_list1=[]
para_list2=[]
labeling = []

ans = []
for x in range(len(train_id)):   #index of PAWS_Wiki training set
    
    for z in range(len(phrase)):  #index of PPDB
        if (phrase[z] in train_sen1[x] and paraphrase[z] in train_sen2[x]):
            print(phrase[z])
            index_list=np.append(index_list, x)
            PPDB_id=np.append(PPDB_id, z)
            sent1=np.append(sent1, train_sen1[x])
            sent2=np.append(sent2, train_sen2[x])
            para_list1=np.append(para_list1, phrase[z])
            para_list2=np.append(para_list2, paraphrase[z])
            labeling = np.append(labeling, train_label[x])

 most valuable 
 's most 
 are generally 
 are generally 
 are generally 
 are generally 
 are generally 
 are generally 
 most popular 
 taken part in 
 even human 
 years later 
 highly significant 
 passed away 
 passed 
 two years 
 previously published 
 some time 
 world 's 
 logistics support 
 resolve the 
 resolve the 
 resolve the 
 secondary schools in 
 only possible 
 to verify 
 and historical 
 and historical 
 very hot 
 since then 
 two years 
 around the world 
 around the world 
 very long 
 very long 
 very nice 
 very nice 
 table of contents 
 , representing the 
 and supported 
 've created 
 later in life 
 well received 
 well received 
 received 
 years later 
 two years 
 second place 
 primary school 
 reached agreement 
 second place 
 nearly four 
 first time 
 very poor 
 two years 
 nuclear power plants 
 nuclear power 
 nuclear power 
 relationships are 
 the southern part 
 are typically 
 are typically 
 are typically 
 are typically 
 are typically 


In [8]:
index_list=[]
PPDB_id = []
sent1=[]
sent2=[]
para_list1=[]
para_list2=[]
labeling = []

ans = []
for x in range(len(dev_id)):   #index of PAWS_Wiki dev set
    
    for z in range(len(phrase)):  #index of PPDB
        if (phrase[z] in dev_sen1[x] and paraphrase[z] in dev_sen2[x]):
            print(phrase[z])
            index_list=np.append(index_list, x)
            PPDB_id=np.append(PPDB_id, z)
            sent1=np.append(sent1, dev_sen1[x])
            sent2=np.append(sent2, dev_sen2[x])
            para_list1=np.append(para_list1, phrase[z])
            para_list2=np.append(para_list2, paraphrase[z])
            labeling = np.append(labeling, dev_label[x])

 only a few months 
 only a few 
 few months 
 center for 
 center for 
 can be seen 
 first time 
 first time 
 is presently 
 is presently 
 is presently 
 is presently 
 is presently 
 is presently 
 presently in 
 5 years 
 several years 
 several years 
 's more 
 relatively rare 
 can only be used 
 members of the 
 a town 
 can be used 
 members of the 
 years later 
 died in 
 two years 
 the east side 
 the east 
 new government 
 is close to 
 artillery fire 
 commonly called 
 ten years 
 ten years 
 to stay 
 birth rate 
 the east side 
 the east 
 received 
 received 
 's work 
 first time 
 first time 
 most commonly 
 are frequently 
 are frequently 
 are frequently 
 are frequently 
 are frequently 
 are frequently 
 most commonly 
 most commonly 
 together with 
 together with 
 there was still 
 there was 
 worth noting 
 several years 
 several years 
 as well as 
 as well as 
 most important 
 most important 
 supervised by 
 there was still 
 there was 
 became par

In [9]:
phrase_dataset = Table().with_column("DEV_PAWS_Wiki_id", index_list, "PPDB_id", PPDB_id, "sent1", sent1, "sent2", sent2, "paraphrase1", para_list1, "paraphrase2", para_list2, "label", labeling)

In [11]:
phrase_dataset.show(4)

DEV_PAWS_Wiki_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,label
23,152079,She was only a few months in French service when she met ...,"She was only in British service for a few months , when ...",only a few months,a few months,0
23,332227,She was only a few months in French service when she met ...,"She was only in British service for a few months , when ...",only a few,few,0
23,1.00491e+06,She was only a few months in French service when she met ...,"She was only in British service for a few months , when ...",few months,"months ,",0
59,1.22026e+06,The temple serves as the cultural and religious center f ...,The temple serves as a cultural and religious centre for ...,center for,centre for,1


In [ ]:
phrase_dataset=phrase_dataset.to_df()
phrase_dataset.to_csv("dataset/Phrase_Level/PAWS_Wiki/PPDB_PAWS_Wiki_DEV.tsv", sep='\t')

In [ ]:
phrase_dataset = Table().with_column("DEV_PAWS_Wiki_id", index_list, "PPDB_id", PPDB_id, "sent1", sent1, "sent2", sent2, "paraphrase1", para_list1, "paraphrase2", para_list2, "label", labeling)

In [ ]:
phrase_dataset.to_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_DEV.csv", sep='\t')

In [ ]:
index_list=[]
PPDB_id = []
sent1=[]
sent2=[]
para_list1=[]
para_list2=[]
labeling = []

ans = []
for x in range(len(test_id)):   #index of PAWS_Wiki test set
    
    for z in range(len(phrase)):  #index of PPDB
        if (phrase[z] in test_sen1[x] and paraphrase[z] in test_sen2[x]):
            print(phrase[z])
            index_list=np.append(index_list, x)
            PPDB_id=np.append(PPDB_id, z)
            sent1=np.append(sent1, test_sen1[x])
            sent2=np.append(sent2, test_sen2[x])
            para_list1=np.append(para_list1, phrase[z])
            para_list2=np.append(para_list2, paraphrase[z])
            labeling = np.append(labeling, test_label[x])

In [1]:
phrase_dataset = Table().with_column("TEST_PAWS_Wiki_id", index_list, "PPDB_id", PPDB_id, "sent1", sent1, "sent2", sent2, "paraphrase1", para_list1, "paraphrase2", para_list2, "label", labeling)
phrase_dataset.to_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_TEST.csv", sep='\t')

NameError: name 'Table' is not defined

In [71]:
dataframe = pd.read_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_TRAIN-Copy1.csv")

In [72]:
dataframe.to_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_TRAIN.tsv", sep='\t', index=False)

In [73]:
dataframe = pd.read_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_DEV-Copy1.csv")
dataframe.to_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_DEV.tsv", sep='\t', index=False)

In [56]:
list1=[]
list2=[]

label_data = Table()
with open("dataset/Phrase_Level/PPDB_PAWS_QQP_DEV.tsv", 'r') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        
        list1.append(row) #pairs start at index 1, with id, setence1, sentence2, label, respectively.
     
    for y in range(6):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        label_data=label_data.with_columns(list1[0][ele], list2[ele])



label_data.show(4)



DEV_PAWS_QQP_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2
3.0,84905.0,"b""Why ca n't countries afford high quality products , bu ...","b""Why ca n't countries afford China 's high quality prod ...",high quality,quality
3.0,129607.0,"b""Why ca n't countries afford high quality products , bu ...","b""Why ca n't countries afford China 's high quality prod ...",high quality,quality
11.0,72492.0,b'How do I start learning electronic music production ? ...,b'How do I begin learning electronic music production ? ...,begin with,start with
11.0,366448.0,b'How do I start learning electronic music production ? ...,b'How do I begin learning electronic music production ? ...,begin with,start with


In [12]:

sent1=phrase_dataset.column("sent1")
sent2=phrase_dataset.column("sent2")
para1=phrase_dataset.column("paraphrase1")
para2=phrase_dataset.column("paraphrase2")

gg = [x.split() for x in sent1]
hh = [x.split() for x in sent2]
ii = [x.split() for x in para1]
jj = [x.split() for x in para2]



22

In [15]:
labels1=[]
for x in range(len(gg)):
    labels1.append(spit_label(gg[x], ii[x], len(ii[x]),len(ii[x]), []))



In [16]:
labels2=[]
for x in range(len(hh)):
    labels2.append(spit_label(hh[x], jj[x], len(jj[x]),len(jj[x]), []))



In [17]:
joined1=[]
for x in labels1:
    joined1.append(" ".join(x))

In [18]:
joined2=[]
for x in labels2:
    joined2.append(" ".join(x))

In [19]:
phrase_dataset=phrase_dataset.with_columns("labels1", joined1, "labels2", joined2)
phrase_dataset.show(4)

DEV_PAWS_Wiki_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,label,labels1,labels2
23,152079,She was only a few months in French service when she met ...,"She was only in British service for a few months , when ...",only a few months,a few months,0,O O B I I E O O O O O O O O O O O O,O O O O O O O B I E O O O O O O O O O O O
23,332227,She was only a few months in French service when she met ...,"She was only in British service for a few months , when ...",only a few,few,0,O O B I E O O O O O O O O O O O O O,O O O O O O O O S O O O O O O O O O O O O
23,1.00491e+06,She was only a few months in French service when she met ...,"She was only in British service for a few months , when ...",few months,"months ,",0,O O O O B E O O O O O O O O O O O O,O O O O O O O O O B E O O O O O O O O O O
59,1.22026e+06,The temple serves as the cultural and religious center f ...,The temple serves as a cultural and religious centre for ...,center for,centre for,1,O O O O O O O O B E O O O O O O O O O,O O O O O O O O B E O O O O O O O O O O


In [20]:

phrase_dataset=phrase_dataset.to_df()
phrase_dataset.to_csv("dataset/Phrase_Level/PAWS_Wiki/LABELED_PAWS_Wiki_DEV.tsv", sep='\t', index=False)

In [34]:
new_Tb

In [14]:
    def spit_label(split_sent, para_split, ori_length, length, listy):
       
    
        if para_split[0]==split_sent[0]:
            
            if ori_length==1:
                listy.append("S")
                for x in range(len(split_sent)-1):
                    listy.append("O")
                return listy    
            
            elif ori_length==length:
                listy.append("B")
                return spit_label(split_sent[1:], para_split[1:], ori_length, length-1, listy)
                
            elif length==1:
                listy.append("E")
                for x in range(len(split_sent)-1):
                    listy.append("O")
                return listy
            
            else:
                listy.append("I")
                return spit_label(split_sent[1:], para_split[1:], ori_length, length-1, listy)
                
        else:
            listy.append("O")
            return spit_label(split_sent[1:], para_split, ori_length, length, listy)

In [88]:
bbb="".join("he ")

In [89]:
bbb

'he '